In [7]:
import sqlalchemy as sa
import pandas as pd
import os

In [4]:
%run ../toxref_mysql_connect.py
print(connect.__doc__)
con = connect(username="",password="", database='', host='')  


    Returns a sqlalchemy instance of engine. 
    Update arguments to connect to local MySQL instance and ToxRefDB.
    


In [5]:
study_ids = pd.read_sql("study",con)["study_id"].unique()

/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [10]:
if "studies" not in os.listdir("."):
    os.mkdir("studies")
    
for study_id in study_ids:
    writer = pd.ExcelWriter(f"studies/STUDYID{study_id}.xlsx")
    pd.read_sql(f"""
    SELECT 
        chemical.chemical_id,
        chemical.dsstox_substance_id,
        chemical.casrn,
        chemical.preferred_name,
        study.study_id,
        study.study_source_id,
        study.study_citation,
        study.study_year,
        study.study_source,
        study.study_type,
        study.study_type_guideline,
        study.species,
        study.strain_group,
        study.strain,
        study.admin_route,
        study.admin_method,
        study.substance_source_name,
        study.substance_purity,
        study.substance_lot_batch,
        study.substance_comment,
        study.dose_start,
        study.dose_start_unit,
        study.dose_end,
        study.dose_end_unit,
        guideline.guideline_number,
        guideline.name,
        guideline.profile_name,
        guideline.description,
        study.study_comment
    FROM 
        chemical INNER JOIN study ON chemical.chemical_id=study.chemical_id 
            LEFT JOIN guideline ON study.guideline_id=guideline.guideline_id
    WHERE study.study_id={study_id}""",con).T.rename(columns={0:f"STUDYID{study_id}"}).to_excel(writer,
                                                                                                merge_cells=False,
                                                                                                sheet_name="study")
    pd.read_sql(f"""
    SELECT
        tg.tg_id,
        tg.sex,
        tg.generation,
        tg.dose_period,
        tg.dose_duration,
        tg.dose_duration_unit,
        tg.n,
        dose.dose_id,
        dose.dose_level,
        dose.conc,
        dose.conc_unit,
        dose.vehicle,
        dtg.dtg_id,
        dtg.dose_adjusted,
        dtg.dose_adjusted_unit,
        dtg.mg_kg_day_value,
        tg.tg_comment,
        dose.dose_comment,
        dtg.dtg_comment
    FROM 
        dose INNER JOIN dtg ON dose.dose_id=dtg.dose_id
            INNER JOIN tg ON tg.tg_id=dtg.tg_id
    WHERE dose.study_id={study_id} AND tg.study_id={study_id}
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="dosedTreatmentGroup")
    pd.read_sql(f"""
    SELECT
        endpoint.endpoint_id,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target,
        effect.effect_id,
        effect.effect_desc,
        tg_effect.effect_desc_free,
        tg_effect.target_site,
        tg_effect.direction,
        tg_effect.life_stage,
        tg.sex,
        tg.generation,
        tg.dose_period,
        tg.dose_duration,
        tg.dose_duration_unit,
        tg.n,
        dose.dose_level,
        dtg.dose_adjusted,
        dtg.dose_adjusted_unit,
        dtg.mg_kg_day_value,
        dtg_effect.treatment_related,
        dtg_effect.critical_effect,
        dtg_effect.sample_size,
        dtg_effect.effect_val,
        dtg_effect.effect_val_unit,
        dtg_effect.effect_var,
        dtg_effect.effect_var_type,
        dtg_effect.time,
        dtg_effect.time_unit,
        dtg_effect.dtg_effect_comment
    FROM 
        endpoint INNER JOIN effect ON endpoint.endpoint_id=effect.effect_id
            INNER JOIN tg_effect ON effect.effect_id=tg_effect.effect_id
                INNER JOIN tg ON tg.tg_id=tg_effect.tg_id
                    INNER JOIN dtg ON tg.tg_id=dtg.tg_id
                        INNER JOIN dose ON dose.dose_id=dtg.dose_id
                            INNER JOIN dtg_effect ON dtg_effect.dtg_id=dtg.dtg_id AND dtg_effect.tg_effect_id=tg_effect.tg_effect_id
    WHERE tg.study_id={study_id} AND dose.study_id={study_id}
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="dosedTreatmentGroupEffect")
    pd.read_sql(f"""
    SELECT 
        endpoint.endpoint_id,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target
    FROM 
        negative_endpoint INNER JOIN endpoint ON endpoint.endpoint_id=negative_endpoint.endpoint_id
    WHERE negative_endpoint.study_id={study_id}
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="negativeEndpoint")
    pd.read_sql(f"""
    SELECT 
        endpoint.endpoint_id,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target,
        effect.effect_id,
        effect.effect_desc
    FROM
        negative_effect INNER JOIN effect ON effect.effect_id=negative_effect.effect_id
            INNER JOIN endpoint ON effect.endpoint_id=endpoint.endpoint_id
    WHERE negative_effect.study_id={study_id}
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="negativeEffect")
    pd.read_sql(f"""
    SELECT
        pod.pod_type,
        pod.qualifier,
        pod.pod_value,
        pod.pod_unit,
        pod.mg_kg_day_value,
        pod.dose_level,
        pod.max_dose_level,
        effect_profile.effect_profile_name,
        effect_profile.effect_profile_description,
        effect_profile_group.group_id,
        effect_profile_group.group_name,
        GROUP_CONCAT(DISTINCT 
            endpoint.endpoint_category,";",
            endpoint.endpoint_type,",",
            endpoint.endpoint_target,",",
            effect.effect_desc SEPARATOR "|") AS effects
    FROM 
        pod INNER JOIN effect_profile ON pod.effect_profile_id=effect_profile.effect_profile_id
            INNER JOIN effect_profile_group ON pod.effect_profile_id=effect_profile_group.effect_profile_id AND pod.group_id=effect_profile_group.group_id
                INNER JOIN pod_tg_effect ON pod.pod_id=pod_tg_effect.pod_id
                    INNER JOIN tg_effect ON tg_effect.tg_effect_id=pod_tg_effect.tg_effect_id
                        INNER JOIN effect ON tg_effect.effect_id=effect.effect_id
                            INNER JOIN endpoint ON endpoint.endpoint_id=effect.endpoint_id
    WHERE pod.effect_profile_id=2 AND pod.study_id={study_id}
    GROUP BY 
        pod.pod_type,
        pod.qualifier,
        pod.pod_value,
        pod.pod_unit,
        pod.mg_kg_day_value,
        pod.dose_level,
        pod.max_dose_level,
        effect_profile.effect_profile_name,
        effect_profile.effect_profile_description,
        effect_profile_group.group_id,
        effect_profile_group.group_name
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="pointOfDeparture")
    pd.read_sql(f"""
    SELECT
        endpoint.endpoint_id,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target,
        effect.effect_id,
        effect.effect_desc,
        tg_effect.effect_desc_free,
        tg_effect.target_site,
        tg_effect.direction,
        tg_effect.life_stage,
        tg.sex,
        tg.generation,
        tg.dose_period,
        tg.dose_duration,
        tg.dose_duration_unit,
        tg.n,
        bmd_ci.doses,
        bmd_ci.ns,
        bmd_ci.means,
        bmd_ci.stdevs,
        bmd_ci.data_type
    FROM 
        bmd_continuous_input AS bmd_ci INNER JOIN tg_effect ON tg_effect.tg_effect_id=bmd_ci.tg_effect_id
            INNER JOIN tg ON tg.tg_id=tg_effect.tg_id
                INNER JOIN effect ON tg_effect.effect_id=effect.effect_id
                    INNER JOIN endpoint ON endpoint.endpoint_id=effect.endpoint_id
    WHERE bmd_ci.study_id={study_id} AND tg.study_id={study_id};
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="BMDContinuousInput")
    pd.read_sql(f"""
    SELECT
        endpoint.endpoint_id,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target,
        effect.effect_id,
        effect.effect_desc,
        tg_effect.effect_desc_free,
        tg_effect.target_site,
        tg_effect.direction,
        tg_effect.life_stage,
        tg.sex,
        tg.generation,
        tg.dose_period,
        tg.dose_duration,
        tg.dose_duration_unit,
        tg.n,
        bmd_di.doses,
        bmd_di.ns,
        bmd_di.incidences,
        bmd_di.data_type
    FROM 
        bmd_dichotomous_input AS bmd_di INNER JOIN tg_effect ON tg_effect.tg_effect_id=bmd_di.tg_effect_id
            INNER JOIN tg ON tg.tg_id=tg_effect.tg_id
                INNER JOIN effect ON tg_effect.effect_id=effect.effect_id
                    INNER JOIN endpoint ON endpoint.endpoint_id=effect.endpoint_id
    WHERE bmd_di.study_id={study_id} AND tg.study_id={study_id}
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="BMDDichotomousInput")
    pd.read_sql(f"""
    SELECT 
        endpoint.endpoint_id,
        endpoint.endpoint_category,
        endpoint.endpoint_type,
        endpoint.endpoint_target,
        effect.effect_id,
        effect.effect_desc,
        tg_effect.effect_desc_free,
        tg_effect.target_site,
        tg_effect.direction,
        tg_effect.life_stage,
        tg.sex,
        tg.generation,
        tg.dose_period,
        tg.dose_duration,
        tg.dose_duration_unit,
        tg.n,
        bmd_m.doses_dropped,
        bmd_m.model_name,
        bmd_m.model_version,
        bmd_m.has_output,
        bmd_m.BMD,
        bmd_m.BMDL,
        bmd_m.BMDU,
        bmd_m.CSF,
        bmd_m.AIC,
        bmd_m.pvalue1,
        bmd_m.pvalue2,
        bmd_m.pvalue3,
        bmd_m.pvalue4,
        bmd_m.Chi2,
        bmd_m.df,
        bmd_m.residual_of_interest,
        bmd_m.warnings,
        bmd_m.logic_bin,
        bmd_m.logic_cautions,
        bmd_m.logic_warnings,
        bmd_m.logic_failures,
        bmd_m.recommended,
        bmd_m.recommended_variable,
        bmd_m.bmr,
        bmd_m.bmr_type
    FROM 
        bmd_models AS bmd_m INNER JOIN tg_effect ON tg_effect.tg_effect_id=bmd_m.tg_effect_id
            INNER JOIN tg ON tg.tg_id=tg_effect.tg_id
                INNER JOIN effect ON tg_effect.effect_id=effect.effect_id
                    INNER JOIN endpoint ON endpoint.endpoint_id=effect.endpoint_id
    WHERE bmd_m.study_id={study_id} AND tg.study_id={study_id}
    """,con).to_excel(writer,merge_cells=False,index=False,sheet_name="BMDModels")
    writer.save()
    print(study_id)

60
63
64
65
67
68
69
70
71
72
73
74
75
76
77
78
80
81
82
83
84
86
87
88
89
90
91
93
94
97
98
99
100
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
133
134
135
136
137
141
142
143
149
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
180
181
182
183
184
185
187
189
190
191
192
193
194
195
196
197
202
204
206
207
211
212
213
214
215
216
217
218
219
220
221
222
223
224
226
227
228
229
230
231
233
236
237
238
239
240
241
242
244
245
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
264
265
267
268
269
270
271
273
274
276
277
278
279
280
281
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
301
302
303
304
305
306
307
308
309
310
311
312
313
316
317
318
319
320
321
322
323
324


/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 21 was cut by GROUP_CONCAT()')
  result = self._query(query)
/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 45 was cut by GROUP_CONCAT()')
  result = self._query(query)
/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 73 was cut by GROUP_CONCAT()')
  result = self._query(query)
/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 101 was cut by GROUP_CONCAT()')
  result = self._query(query)


325


/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 40 was cut by GROUP_CONCAT()')
  result = self._query(query)


326
327
328
330
332
333
336
338
339
340
341
342
343
344
346
347
348
349
350
351
352
354
355
356
357
359
362
363
364
366
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
388
389
390
391
392
393
395
396
398
399
401
403
404
405
406
407
408
409
410
411
412
413
418
420
422
424
425
426
428
429
431
432
433
435
436
437
438
439
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
472
473
474
475
476
477
478
479
480
481
482
483
486
487
488
489
492
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
513
514
515
516
517
518
519
520
521
523
524
526
527
528
529
534
541
542
543
544
546
547
549
550
552
553
554
555
557
558
559
560
561
562
563
564
565
566
567
571
572
573
574
575
576
577
578
579
580
581
583
585
586
587
588
589
590
593
594
596
597
598
599
600
601
602
603
604
605
606
608
611
612
614
616
617
618
619
621
622
625
626
627
628
629
630
631
632
634
635
636
639
643
644
650
653
654
656
657
659


...

7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266


/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 46 was cut by GROUP_CONCAT()')
  result = self._query(query)
/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 100 was cut by GROUP_CONCAT()')
  result = self._query(query)


7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285
7286
7287
7288
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302


/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 49 was cut by GROUP_CONCAT()')
  result = self._query(query)
/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1260, 'Row 122 was cut by GROUP_CONCAT()')
  result = self._query(query)


7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398
7399
7400
7401
7402
7403
7404
7405
7406
7407
7408
7409
7410
7411
7412
7413
7414
7415
7416
7417
7418
7419
7420
7421
7422
7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
